In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

from pathlib import Path
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import RFE

from utils import prepare_dataset
import spikesorters as ss

matplotlib.use('qt4Agg')

/home/mclancy/.local/share/pyenv/versions/3.8.3/lib/python3.8/importlib/__init__.py:127: MatplotlibDeprecationWarning: 
The matplotlib.backends.backend_qt4agg backend was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  return _bootstrap._gcd_import(name[level:], package, level)


In [3]:
os.environ["TMP"] = "/home/mclancy/SpikeConfidence/.tmp/"
os.environ["TMPDIR"] = "/home/mclancy/SpikeConfidence/.tmp/"
os.environ["TEMPDIR"] = "/home/mclancy/SpikeConfidence/.tmp/"
os.environ["ML_TEMPORARY_DIRECTORY"] = "/home/mclancy/SpikeConfidence/.tmp"

base_dir = Path('/home/mclancy/SpikeConfidence/')
analyses_dir = base_dir / 'analyses'
spike_sorter_dir = base_dir / 'spikesorters'

# kilosort2_5
ss.Kilosort2_5Sorter.set_kilosort2_5_path(spike_sorter_dir / 'Kilosort')

ss.IronClustSorter.set_ironclust_path(spike_sorter_dir / 'ironclust')

dataset_path = Path("/home/mclancy/SpikeConfidence/analyses/recordings_50cells_SqMEA_2020/recordings_50cells_SqMEA-10-15_600.0_10.0uV_21-01-2020_18-12.h5")
dataset_path_2 = Path("/home/mclancy/SpikeConfidence/analyses/recordings_50cells_SqMEA/recordings_50cells_SqMEA-10-15um_60.0_10.0uV_27-03-2019_13-31-005.h5")
# session = SpikeSorter(dataset_path)
# recording = session.get_recording()
# ground_truth = session.get_ground_truth()

# sorting = session.get_sorting(sorter_name='herdingspikes')

# sorter_names = ['klusta', 'tridesclous', 'mountainsort4',
#                 'ironclust', 'kilosort', 'kilosort2', 'kilosort2_5',
#                 'waveclus']

Setting KILOSORT2_5_PATH environment variable for subprocess calls to: /home/mclancy/SpikeConfidence/spikesorters/Kilosort
Setting IRONCLUST_PATH environment variable for subprocess calls to: /home/mclancy/SpikeConfidence/spikesorters/ironclust


/home/mclancy/.local/share/pyenv/versions/SpikeConfidence/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
# Spyking circus causes d_prime to break
sorter_names = ['mountainsort4']#],['herdingspikes', 'spykingcircus']#'kilosort2_5',['ironclust','tridesclous',

metric_names = np.array(["num_spikes", "firing_rate", "presence_ratio",
                "isi_violation", "amplitude_cutoff", "snr",
                "max_drift", "cumulative_drift", "silhouette_score",
                "isolation_distance", "l_ratio",
                "nn_hit_rate", "nn_miss_rate","d_prime"])

dataset_paths = [dataset_path, dataset_path_2]

/home/mclancy/.local/share/pyenv/versions/SpikeConfidence/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
# Preparing dataset

X, y = prepare_dataset(dataset_paths=dataset_paths,
                       sorter_names=sorter_names, metric_names=metric_names
                       )

#Standard scalar shouldn't affect Logistic regression, but it does in this case (for mountainsort4)
model = make_pipeline(StandardScaler(), LogisticRegression())
# Cross validation code
# cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
# score = cross_val_score(model, X, y, cv=cv)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

scaler = StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

# False positive classification via logistic regression
model.fit(X_train_transformed, y_train)
model_accuracy = model.score(X_test_transformed, y_test)
print(f"Full metrics model accuracy is {model_accuracy}")

ValueError: n_components cannot be larger than min(n_features, n_classes - 1).

In [6]:
ranked_metrics_index = sorted(range(len(metric_names)), key=lambda k: abs(model.named_steps.logisticregression.coef_[0][k]), reverse=True)

/home/mclancy/.local/share/pyenv/versions/SpikeConfidence/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


For the k most important features, plot the training accuracies.
As the input data is standardised, the coefficients can be ranked by their absolute value.

We can retrain the logistic regression with the reduced metric set,
and as a result can assess the redundancy of the metrics.

In [16]:
accuracies = []
for i in range(1, len(ranked_metrics_index)+1):
    m_names = metric_names[ranked_metrics_index[:i]]
    print(f"Features are {m_names}")

    X, y = prepare_dataset(dataset_paths=dataset_paths, metric_names=m_names, sorter_names=sorter_names)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

    scaler = StandardScaler().fit(X_train)
    X_train_transformed = scaler.transform(X_train)
    X_test_transformed = scaler.transform(X_test)

    # False positive classification via logistic regression
    model = make_pipeline(scaler, LogisticRegression())
    model.fit(X_train_transformed, y_train)
    y_preds = model.predict(X_test_transformed)
    model_f1_score = f1_score(y_test, y_preds, average='binary')
    accuracies.append(model_f1_score)
    print(f"F1 Score is {model_f1_score}")

Features are ['l_ratio']
F1 Score is 0.8702634292752892
Features are ['l_ratio' 'presence_ratio']
F1 Score is 0.8962370736136516
Features are ['l_ratio' 'presence_ratio' 'max_drift']
F1 Score is 0.8791709634629538
Features are ['l_ratio' 'presence_ratio' 'max_drift' 'snr']
F1 Score is 0.9045381077245596
Features are ['l_ratio' 'presence_ratio' 'max_drift' 'snr' 'cumulative_drift']
F1 Score is 0.900474306946799
Features are ['l_ratio' 'presence_ratio' 'max_drift' 'snr' 'cumulative_drift'
 'nn_hit_rate']
F1 Score is 0.9047007979704116
Features are ['l_ratio' 'presence_ratio' 'max_drift' 'snr' 'cumulative_drift'
 'nn_hit_rate' 'nn_miss_rate']
F1 Score is 0.9002971821773101
Features are ['l_ratio' 'presence_ratio' 'max_drift' 'snr' 'cumulative_drift'
 'nn_hit_rate' 'nn_miss_rate' 'firing_rate']
F1 Score is 0.9002971821773101
Features are ['l_ratio' 'presence_ratio' 'max_drift' 'snr' 'cumulative_drift'
 'nn_hit_rate' 'nn_miss_rate' 'firing_rate' 'num_spikes']
F1 Score is 0.9002971821773101


In [ ]:
plt.clf()
plt.plot(range(len(metric_names)), accuracies)
plt.suptitle("Logistic regression performances with best k features")
plt.xlabel('k (num features)')
plt.ylabel('F1 Score')
plt.xticks(np.arange(1, len(metric_names)+1))
plt.yticks(np.arange(0.84, 0.95, 0.01))